In [9]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Show code: <a href="javascript:code_toggle()">here</a>.''')

In [2]:
import numpy as np
import pandas as pd
import requests
import ipywidgets as widgets
from sklearn import tree

In [3]:
# get trainingsdata
print('Get protein binding trainings data...')
binding = pd.read_csv('data/binding.csv',header=None, names=['p1','p2','y'])

Get protein binding trainings data...


In [4]:
# get protein sequences from Uniprot DB
print('Get protein sequences from Uniprot DB...')
def get_seq_fastna(df,colname):
    seq = []
    for i in df[colname]:
        link = "https://www.uniprot.org/uniprot/"+i+".fasta"
        seq.append(requests.get(link).text)
    df[colname+"_fasta"] = seq
    return df

binding = get_seq_fastna(df=binding,colname='p1')
binding = get_seq_fastna(df=binding,colname='p2')

Get protein sequences from Uniprot DB...


In [5]:
# create amino acid features
print('Creating amino acid features...')

def create_aminoacid_occurence_counts(df,colname):
    aminoacid_codes = ['G','A','L','M','F','W','K','Q','E','S','P','V','I','C','Y','H','R','N','D','T']
    occ = []
    for i in aminoacid_codes:
        df[colname+'_cnt_'+i]=df[colname].str.count(i)
    return df

binding = create_aminoacid_occurence_counts(binding,colname='p1_fasta')
binding = create_aminoacid_occurence_counts(binding,colname='p2_fasta')

Creating amino acid features...


In [6]:
# training ML model
print('Training ML model...')
clf = tree.DecisionTreeClassifier()
clf = clf.fit(binding.iloc[:,6:], binding.loc[:,'y'])

Training ML model...


# Remissio UG
##### Predicting side effects

In [11]:
print('Protein sequence immunotherapy receptor:')
input_p1 = widgets.Text()
display(input_p1)
print('Protein sequence healthy cell:')
input_p2 = widgets.Text()
display(input_p2)

def handle_submit(sender):
    binding_pred = pd.DataFrame({'p1':[input_p1.value], 'p2':[input_p2.value]})
    binding_pred = get_seq_fastna(df=binding_pred,colname='p1')
    binding_pred = get_seq_fastna(df=binding_pred,colname='p2')
    binding_pred = create_aminoacid_occurence_counts(binding_pred,colname='p1_fasta')
    binding_pred = create_aminoacid_occurence_counts(binding_pred,colname='p2_fasta')
    #print('Checking for binding of protein '+input_p1.value+' and '+input_p2.value)
    y_pred = clf.predict(binding_pred.iloc[:,5:])[0]
    if y_pred ==0:
        print('Result: No side-effects')
    else:
        print('You ve just discovered a life-saving T-cell target!')

input_p2.on_submit(handle_submit)

Protein sequence immunotherapy receptor:


Text(value='')

Protein sequence healthy cell:


Text(value='')